**Notas para contenedor de docker:**

Comando de docker para ejecución de la nota de forma local:

nota: cambiar `<ruta a mi directorio>` por la ruta de directorio que se desea mapear a `/datos` dentro del contenedor de docker.

```
docker run --rm -v <ruta a mi directorio>:/datos --name jupyterlab_numerical -p 8888:8888 -d palmoreck/jupyterlab_numerical:1.1.0
```

password para jupyterlab: `qwerty`

Detener el contenedor de docker:

```
docker stop jupyterlab_numerical
```


Documentación de la imagen de docker `palmoreck/jupyterlab_numerical:1.1.0` en [liga](https://github.com/palmoreck/dockerfiles/tree/master/jupyterlab/numerical).

---

Nota generada a partir de [liga](https://www.dropbox.com/s/fyqwiqasqaa3wlt/3.1.1.Multiplicacion_de_matrices_y_estructura_de_datos.pdf?dl=0), [liga2](https://www.dropbox.com/s/l4hq45rj860ql9f/3.1.2.Localidad_y_vectorizacion.Analisis_del_error_en_computos_matriciales_basicos.pdf?dl=0)

# 3.1 El cómputo matricial y el álgebra lineal. Vectorización, BLAS y el uso del caché eficientemente.

El cómputo matricial está construído sobre una jerarquía de operaciones del álgebra lineal:

* Productos punto involucran operaciones escalares de suma y multiplicación.

* La multiplicación matriz-vector está hecha de productos punto.

* La multiplicación matriz-matriz utiliza colecciones de productos matriz-vector.

Las operaciones anteriores se describen en el álgebra lineal con la teoría de espacios vectoriales pero también es posible describirlas en una forma algorítmica. Ambas descripciones se complementan una a la otra.

Manejaremos nombres que en [Linear Algebra Package: LAPACK](http://www.netlib.org/lapack/explore-html/dir_fa94b7b114d387a7a8beb2e3e22bf78d.html) son utilizados para denotar algunas operaciones con escalares, vectores o matrices. Ver [ Reference-LAPACK / lapack](https://github.com/Reference-LAPACK/lapack) para su github.

## Operación del producto interno estándar o producto punto

Consideramos $x,y \in \mathbb{R}^n$. El producto punto entre $x$ y $y$ es $c = x^Ty = \displaystyle \sum_{i=1}^n x_iy_i$. 

**Ejemplo y algoritmo del producto punto:**

In [2]:
c=0
n=5
x=[-1]*n
y=[1.5]*n

for i in range(n):
    c += x[i]*y[i]

In [3]:
c

-7.5

**Obs:**

* El producto punto de dos $n$-vectores involucran $n$ multiplicaciones y $n$ sumas para un total de $2n$ operaciones. Usamos la notación $\mathcal{O}(\cdot)$ para escribir que el producto punto es $\mathcal{O}(n)$ y se lee "de orden $n$ o proporcional a $n$" para indicar que la **cantidad de trabajo** tiene un comportamiento **lineal** con la dimensión $n$. También tal cantidad de trabajo opera sobre una **cantidad lineal de datos**.

## Operación **saxpy**

Consideramos $\alpha \in \mathbb{R}, x,y \in \mathbb{R}^n$. El nombre lo recibe por *scalar alpha x plus y*. En LAPACK se escribe en forma *update*:

$$y=\alpha x + y \therefore y_i = \alpha x_i + y_i \forall i=1,...,n$$

**Obs:** 

* El símbolo $=$ no se utiliza como igualdad de expresiones sino como en computación para denotar asignación.

* También encontramos en LAPACK `caxpy` o `daxpy` para el caso complejo y para números en doble precisión respectivamente.

* Ésta operación realiza un trabajo de $\mathcal{O}(n)$ sobre una cantidad de datos $\mathcal{O}(n)$.

**Ejemplo y algoritmo de saxpy:**

In [11]:
alpha=2
n=5
x=[-2]*n
y=[0]*n

for i in range(n):
    y[i] += alpha*x[i]

In [12]:
y

[-4, -4, -4, -4, -4]

o en una forma *update*:

In [5]:
alpha=2
n=5
x=[-2]*n
y=[3,4,-1,0,1]

for i in range(n):
    y[i] += alpha*x[i]

In [6]:
y

[-1, 0, -5, -4, -3]

## Multiplicación matriz-vector

Consideramos $A \in \mathbb{R}^{m \times n}, x \in \mathbb{R}^n, y \in \mathbb{R}^m$. La operación $y = y + Ax$ es una operación *generalizada* saxpy, por ello se denomina **gaxpy** pero en LAPACK podemos encontrar esta operación con nombres como [sgemv](http://www.netlib.org/lapack/explore-html/db/d58/sgemv_8f.html), [dgemv](http://www.netlib.org/lapack/explore-html/dc/da8/dgemv_8f.html), [cgemv](http://www.netlib.org/lapack/explore-html/d4/d8a/cgemv_8f.html) o [zgemv](http://www.netlib.org/lapack/explore-html/db/d40/zgemv_8f.html) para los casos de precisión simple, doble o números complejos. Hay diferentes formas de visualizar y escribir el algoritmo. Por ejemplo para una matriz $A$ con entradas:


In [19]:
m=2
n=5
A=[[1.2]*n if i%2==0 else [1]*n for i in range(m)]

In [20]:
A

[[1.2, 1.2, 1.2, 1.2, 1.2], [1, 1, 1, 1, 1]]

In [31]:
A[0][0]

1.2

In [33]:
A[1][n-1]

1

se tiene:

### Algoritmo gaxpy *row oriented*

In [27]:
x=[2]*n
y=[0]*m
for i in range(m):
    for j in range(n):
        y[i]+=A[i][j]*x[j]


In [28]:
y

[12.0, 10]

o en forma update:

In [29]:
x=[2]*n
y=[-1]*m
for i in range(m):
    for j in range(n):
        y[i]+=A[i][j]*x[j]


In [30]:
y

[11.0, 9]

**Comentarios:**

* En la versión *row oriented* del algoritmo *gaxpy*, el *inner loop* realiza productos punto entre el $i$-ésimo renglón de $A$ y el vector $x$. Se realizan $m$ productos punto.

* El acceso a la matriz $A$ es por renglón.

También puede considerarse al algoritmo *gaxpy* en una forma orientada por columnas:

### Algoritmo gaxpy *column oriented*

Para este algoritmo visualizamos al producto matriz-vector como una combinación lineal de las columnas de $A$:

$$Ax = \displaystyle \sum_{j=1}^n a_jx_j$$

con $a_j$ la $j$-ésima columna de $A$.


In [34]:
x=[2]*n
y=[0]*m
for j in range(n):
    for i in range(m):
        y[i]+=A[i][j]*x[j]

In [35]:
y

[12.0, 10]

**Obs:**

* El algoritmo de multiplicación matriz-vector (versión *row* o *column* oriented) involucra $\mathcal{O}(mn)$ operaciones o una cantidad **cuadrática** de trabajo, que podemos entender como "si duplicamos cada dimensión de $A$ entonces la cantidad de trabajo se incrementa por un factor de $4$". Además este número de operaciones trabajan sobre una matriz o sobre una cantidad **cuadrática** de datos.

* La versión *column oriented* se puede analizar desde el punto de vista puramente algorítmico como un intercambio entre las líneas con los índices $i$ y $j$ de cada *loop* y un acceso a los datos de la matriz por columna. O bien, se puede analizar desde el álgebra lineal indicando que el vector $y$ está en el **espacio generado** por las columnas de $A$ y cuyas coordenadas están siendo dadas por las entradas del vector $x$:

<img src="https://dl.dropboxusercontent.com/s/6a2b7rjs4a71sni/combinacion_lineal_columnas_A.png?dl=0" heigth="700" width="700">


**Referencias:**

* E. Anderson, Z. Bai, C. Bischof, L. S. Blackford, J. Demmel, J. Dongarra, J. Du Croz,
A. Greenbaum, S. Hammarling, A. Mckenney and D. Sorensen, LAPACK Users Guide, Society for Industrial and Applied Mathematics, Philadelphia, PA, third ed., 1999.

* G. H. Golub, C. F. Van Loan, Matrix Computations, John Hopkins University
Press, 2013.